# Laborator 09

În cadrul laboratorului vom studia cum pot fi utilizați algoritmii evolutivi pentru a approxima necunoscutele într-un sistem de ecuații liniare.

Începem prin instalarea DEAP, folosid comanda `pip install`:

In [ ]:
!pip install deap

     |████████████████████████████████| 163kB 5.4MB/s 


Importarea componentelor și librăriilor principale:

In [ ]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import numpy as np
import random
import array

# Sistem de două ecuații cu două necuoscute 

Fie sistemul de ecuații lineare: 

$\begin{cases} x + 2y = 4 \\ 4x + 4y = 12 \end{cases}$

Avem nevoie de o funcție, care pentru niște valori presupuse, evaluează eroarea (diferența dintre valoarea reală și approximație:

In [ ]:
def error(genes):
  x = genes[0] 
  y = genes[1]
  err1 = x + 2 * y - 4
  err2 = 4 * x + 4 * y - 12
  totalErr = abs(err1) + abs(err2)
  return totalErr, 
  #pentru ca va fi folosita ca functie fitness, rezultatul returnat trebuie sa fie iterabil 
  #chiar daca avem o singura valoare, punem si o virgula

Testare:

In [ ]:
print(error([0,0]))
print(error([1,2]))
print(error([2,1]))

(16,)
(1,)
(0,)


Într-un prim pas, definim reprezenatrea soluțiilor, strategia de selecție și funcția fitness.

In [ ]:
# dorim să minimizăm funcția erroare
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# reprezenatre - un șir (array) de numere reale (double)
creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Strategy' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Definim lungimea "cromosomului", a soluției. Pentru exemplul actual $x, y$, 2 numere reale:

In [ ]:
IND_SIZE = 2

Definim funcțiile ce facilitează utilizarea reprezentării alese pentru a genera indivizi (în algoritmul evolutiv) și a forma o populație. 

In [ ]:
def generateES(individual, strategy, size):
    ind = individual(np.random.normal() for _ in range(size))
    ind.strategy = strategy(np.random.normal() for _ in range(size))
    return ind

toolbox = base.Toolbox()

# functii pt. generarea indivizilor si a populatiei
toolbox.register("individual", generateES, creator.Individual, creator.Strategy, IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Parametrizăm algoritmul, spețificând operatorii predefiniți pe care dorim să îi foolosim: https://deap.readthedocs.io/en/master/api/algo.html

In [ ]:
# parametrii pt. alg algorithms.eaMuCommaLambda

toolbox.register("evaluate", error)

toolbox.register("mate", tools.cxESBlend, alpha=0.1)
toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.3)
toolbox.register("select", tools.selTournament, tournsize=4)

Cu ajutorul modului `Statistics` putem genera statisticile (și afișsa) aferente prodesului de optimizare.

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# reținem cele mai bune 10 soluții
hof = tools.HallOfFame(10)

In [ ]:
pop = toolbox.population(n=100)

pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=100, lambda_=200, 
            cxpb=0.6, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg    	std    	min    	max    
0  	100   	16.8139	7.72415	1.45137	32.9762
1  	164   	10.0517	4.40583	1.01326	21.8522
2  	155   	6.0827 	2.62817	0.274307	12.0913
3  	151   	3.07223	1.71604	0.508477	8.72251
4  	151   	1.49164	0.854447	0.407305	4.6175 
5  	153   	0.786355	0.448558	0.166141	2.5176 
6  	161   	0.48765 	0.175657	0.166141	1.09117
7  	160   	0.344442	0.0889232	0.166141	0.634781
8  	157   	0.286726	0.0490464	0.178843	0.469366
9  	161   	0.242109	0.0502047	0.168423	0.434055
10 	148   	0.218548	0.0382467	0.17351 	0.382836
11 	163   	0.201177	0.0312854	0.170973	0.334853
12 	158   	0.178435	0.00943072	0.16601 	0.223744
13 	156   	0.172507	0.00341392	0.165259	0.196071
14 	151   	0.170369	0.00245236	0.164841	0.176428
15 	156   	0.169521	0.00257599	0.164841	0.174863
16 	163   	0.167984	0.00244201	0.164841	0.171885
17 	158   	0.166172	0.00157642	0.164481	0.172864
18 	171   	0.165121	0.000661757	0.164084	0.167354
19 	154   	0.164631	0.000259648	0.164187	0.165357
20 	159   	0

Afișsarea celor mai bune soluții:

In [ ]:
for h in hof:
  print('Solution [x={}, y={}]  has fitness {}'.format(h[0], h[1], h.fitness))

Solution [x=2.003184615127936, y=0.9968153848720639]  has fitness (0.003184615127936219,)
Solution [x=2.003184615127936, y=0.9968153848720638]  has fitness (0.003184615127936219,)
Solution [x=2.003184615127936, y=0.9968153848720637]  has fitness (0.003184615127936219,)
Solution [x=2.0031846151279367, y=0.9968153848720634]  has fitness (0.003184615127936219,)
Solution [x=2.0031846151279367, y=0.9968153848720636]  has fitness (0.003184615127936219,)
Solution [x=2.003184615127937, y=0.9968153848720631]  has fitness (0.003184615127936663,)
Solution [x=2.0031846151279367, y=0.9968153848720633]  has fitness (0.003184615127936663,)
Solution [x=2.003184615127936, y=0.9968153848720636]  has fitness (0.003184615127936663,)
Solution [x=2.003184615127937, y=0.9968153848720628]  has fitness (0.003184615127937107,)
Solution [x=2.003184615127937, y=0.9968153848720629]  has fitness (0.003184615127937107,)


# Exerciții

1. Approximați (folosind un algoritm evolutiv) soluțiile sistemelor de ecuații:

  a)  $\begin{cases} 2x + y - 2z = -1 \\ 3x - 3y -z = 5  \\ x  - 2y + 3z = 6 \end{cases}$

  b)  $\begin{cases} x + y +z = 7 \\ 3x - 2y -z = 4  \\ x  + 6y + 5z = 24 \end{cases}$


2. Generalizați procesul de approximare astfel încât să putem rezolva sisteme lineare de ecuații cu un număr arbitrar de variabile. Observați, cum numărul de variabile și ecuații este egal cu `sqrt(len(genes))`: pentru 2 variabile și 2 ecuații `len(genes) = 4`, pentru  3 variabile și 3 ecuații `len(genes) = 9`,  `k` variabile și `k` ecuații `len(genes) = k*k`. 



In [2]:
def err1(genes):
  x = genes[0] 
  y = genes[1]
  z = genes[2]
  errX = 2* x  +  y  - 2 * z + 1
  errY = 3 * x - 3 * y - z - 5
  errZ = 3 * x - 3 * y - z - 5

In [8]:
  totalErr = abs(errX) + abs(errY) + abs(errZ)
  return totalErr 
  #pentru ca va fi folosita ca functie fitness, rezultatul returnat trebuie sa fie iterabil 
  #chiar daca avem o singura valoare, punem si o virgula

NameError: ignored

In [7]:
print(err1([0,0,0]))
print(err1([1,2,3]))
print(err1([2,1,3]))

None
None
None


In [5]:
def err2(genes):
  x = genes[0] 
  y = genes[1]
  z = genes[2]
  errX2 = x  +  y + z - 7
  errY2 = 3 * x - 2 * y - z - 4
  errZ2 = x + 6 * y + 5 * z - 24

In [ ]:
totalErr2 = abs(errX2) + abs(errY2) + abs(errZ2)
  return totalErr2, 
  #pentru ca va fi folosita ca functie fitness, rezultatul returnat trebuie sa fie iterabil 
  #chiar daca avem o singura valoare, punem si o virgula

In [ ]:
print(err2([0,0,0]))
print(err2([1,2,3]))
print(err2([2,1,3]))